# Bernstein-Vazirani algorithm（Overview）

We explain Bernstein-Vazirani algorithm.

In this algorithm, we consider the following function that takes $x$ as input and computes the output using the internal bit sequence $a$.

$$
f_a(x) = (a\cdot x) \bmod2 = (\sum_i a_i x_i) \bmod2
$$

The goal is to find the bit sequence $a$ from the output of $f_a(x)$.

$a, x$ are $n$-bit strings $a = a_0 a_1... a_{n-1}$, $x = x_0 x_1.... x_{n-1}$, respectively.

Consider the case where $a$ is a 4-bit sequence $1001$.  
A foolproof way to find $x$ in classical calculations is to input $x$ in order, with only one bit set to 1 and the others to 0, as in $x=1000, 0100, 0010, 0001$.   
From $0 \cdot 0 = 0 \cdot 1 = 0$, the bit with $x_i=0$ do not affect the result, so we can determine the value of $a_i$ one bit at a time.

Calculate $f_a(x)$ where $a=1001$ for four different $x$ as follows.

$(1001 \cdot 1000) \bmod2 = 1$  
$(1001 \cdot 0100) \bmod2 = 0$  
$(1001 \cdot 0010) \bmod2 = 0$  
$(1001 \cdot 0001) \bmod2 = 1$

From the above output, we can get $a=1001$.

The Bernstein-Vazirani algorithm uses quantum superposition to find $a$ in a single calculation.  

The specific circuit for the case $a=1001$ is as follows.   

<img src="../tutorial-ja/img/102_img.png" width="30%">

Let the $CX$ gate act on the qubits $i$, where $a_i=1$ are the control qubits, respectively.  
All target qubits are the fourth auxiliary qubit.

Let's check the states.

$$
\begin{align}
\lvert \psi_1\rangle &= \biggl(\otimes^4 H\lvert 0\rangle \biggr)\otimes H \lvert 1\rangle \\
&= \frac{1}{\sqrt{2}}(\lvert 0\rangle + \lvert 1\rangle) \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle + \lvert 1\rangle) \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle + \lvert 1\rangle) \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle + \lvert 1\rangle) \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle - \lvert 1\rangle)
\end{align}
$$

Next, check the state of $\lvert \psi_2\rangle$  
Let's focus only on the 0th and 4th qubits that the $CX$ gates act on.

$$
\begin{align}
\lvert \psi_1\rangle_{04} &= \frac{1}{\sqrt{2}}(\lvert 0\rangle_0 + \lvert 1\rangle_0) \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle_4 - \lvert 1\rangle_4) \\
&= \lvert +\rangle_0 \otimes \lvert -\rangle_4
\end{align}
$$

$$
\begin{align}
\lvert \psi_2\rangle_{04} &= \frac{1}{\sqrt{2}}\lvert 0\rangle_0 \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle_4 - \lvert 1\rangle_4) + \frac{1}{\sqrt{2}}\lvert 1\rangle_0 \otimes \frac{1}{\sqrt{2}}(\lvert 1\rangle_4 - \lvert 0\rangle_4) \\
&= \frac{1}{\sqrt{2}}\lvert 0\rangle_0 \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle_4 - \lvert 1\rangle_4) - \frac{1}{\sqrt{2}}\lvert 1\rangle_0 \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle_4 - \lvert 1\rangle_4) \\
&= \frac{1}{\sqrt{2}}(\lvert 0\rangle_0 - \lvert 1\rangle_0 ) \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle_4 - \lvert 1\rangle_4) \\
&=  \lvert -\rangle_0 \otimes \lvert -\rangle_4
\end{align}
$$



By acting on the 0th and 4th qubits with a $CX$ gate, we inverted the phase of the 0th qubit, changing it to $\lvert + \rangle \to \lvert - \rangle$.    
Similarly, with $CX$ gates acting on the third and fourth qubits, the state $\lvert\psi_3\rangle$ is as follows

$$
\lvert \psi_3\rangle = \frac{1}{\sqrt{2}}(\lvert 0\rangle - \lvert 1\rangle) \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle + \lvert 1\rangle) \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle + \lvert 1\rangle) \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle - \lvert 1\rangle) \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle - \lvert 1\rangle)
$$

Finally, let the H-gate act.

$$
\lvert \psi_4\rangle = \lvert 1\rangle \otimes \lvert 0\rangle \otimes \lvert 0\rangle \otimes \lvert 1\rangle \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle - \lvert 1\rangle)
$$

Measure the 0th to third qubits and we get $1001$ as the measurement result.  
Now we can find $a$.

Let's implement this with blueqat.

In [1]:
from blueqat import Circuit
import numpy as np

First, prepare a function to act oracle $U_f$.

In [ ]:
def Uf(c, a):
    N = len(a)
    for i, val in enumerate(list(a)):
        if val == '1':
            c.cx[i, len(a)]

The following is the main body of the algorithm.  
First, determine the $a$ you want to find by a random number.

Find $a$ from the output result of the quantum circuit using the oracle, and check if the answer is correct.

In [67]:
n = 4

a = ''
for i in range(n):
    a += str(np.random.randint(2))
    
c = Circuit(n + 1)
c.x[n].h[:]
Uf(c, a)
c.h[:].m[:]
res = c.run(shots = 1000)

print(res)

if [arr[:n] for arr in res.keys()] == [a]:
    print("OK")
else:
    print("incorrect")

Counter({'11001': 1000})
OK


From the above, we were able to find the $a$ that the oracle has internally by using the Bernstein-Vazirani algorithm.